In [2]:
import pandas as pd

df = pd.read_csv("Resources/diabetes_prediction_dataset.csv")

# creating a new variable for the clean dataframe
clean_df = df

# converting hypertension, heart_disease, and diabetes columns to boolean values
# people either have them, or they don't
clean_df["hypertension"] = clean_df["hypertension"].replace({0: False, 1: True})
clean_df["heart_disease"] = clean_df["heart_disease"].replace({0: False, 1: True})
clean_df["diabetes"] = clean_df["diabetes"].replace({0: False, 1: True})


clean_df

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,False,True,never,25.19,6.6,140,False
1,Female,54.0,False,False,No Info,27.32,6.6,80,False
2,Male,28.0,False,False,never,27.32,5.7,158,False
3,Female,36.0,False,False,current,23.45,5.0,155,False
4,Male,76.0,True,True,current,20.14,4.8,155,False
...,...,...,...,...,...,...,...,...,...
99995,Female,80.0,False,False,No Info,27.32,6.2,90,False
99996,Female,2.0,False,False,No Info,17.37,6.5,100,False
99997,Male,66.0,False,False,former,27.83,5.7,155,False
99998,Female,24.0,False,False,never,35.42,4.0,100,False


In [71]:
# look at duplicate rows
duplicates = clean_df[clean_df.duplicated()]

duplicates.count()

gender                 3848
age                    3848
hypertension           3848
heart_disease          3848
smoking_history        3848
bmi                    3848
HbA1c_level            3848
blood_glucose_level    3848
diabetes               3848
dtype: int64

In [53]:
# dropping na values
clean_df = clean_df.dropna()

# checking values for smoking history
clean_df.groupby("smoking_history")["smoking_history"].count()

smoking_history
No Info        35816
current         9286
former          9352
never          35095
not current     6447
Name: smoking_history, dtype: int64

In [54]:
# removing rows where smoking history is "ever"
clean_df = clean_df[clean_df["smoking_history"] != "ever"]

# checking remaining rows
clean_df.count()

gender                 95996
age                    95996
hypertension           95996
heart_disease          95996
smoking_history        95996
bmi                    95996
HbA1c_level            95996
blood_glucose_level    95996
diabetes               95996
dtype: int64

In [7]:
# viewing the current clean_df
clean_df

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,False,True,never,25.19,6.6,140,False
1,Female,54.0,False,False,No Info,27.32,6.6,80,False
2,Male,28.0,False,False,never,27.32,5.7,158,False
3,Female,36.0,False,False,current,23.45,5.0,155,False
4,Male,76.0,True,True,current,20.14,4.8,155,False
...,...,...,...,...,...,...,...,...,...
99995,Female,80.0,False,False,No Info,27.32,6.2,90,False
99996,Female,2.0,False,False,No Info,17.37,6.5,100,False
99997,Male,66.0,False,False,former,27.83,5.7,155,False
99998,Female,24.0,False,False,never,35.42,4.0,100,False


In [56]:
# creating a summary table comapring age, bmi, HbA1c levels, and blood glucose levels between people who have and don't have diabetes
summary_1 = clean_df.groupby("diabetes")[["age", "bmi", "HbA1c_level", "blood_glucose_level"]].agg(["mean", "median", "std"])

# transposing the dataframe so it is easier to read
summary_1.transpose()

diabetes                         False       True 
age                 mean     39.809312   61.023418
                    median   40.000000   62.000000
                    std      22.444480   14.601648
bmi                 mean     26.828584   31.995230
                    median   27.320000   29.950000
                    std       6.376896    7.580808
HbA1c_level         mean      5.396781    6.934716
                    median    5.800000    6.600000
                    std       0.971959    1.075827
blood_glucose_level mean    132.860597  194.387020
                    median  140.000000  160.000000
                    std      34.233557   58.713637